Features for Meta-Learner in Stacking Ensemble

In [1]:
from pathlib import Path
import os
import sys
import gc
import shutil
import json
import math
import uuid
from collections import defaultdict
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from tqdm import tqdm
import matplotlib.pyplot as plt
from typing import Dict, List, Tuple, NamedTuple, Callable, Iterable, Set, Optional, Any
import textstat
import scml
from scml import pandasx as pdx
import lalaes2 as mylib

In [2]:
version = "01"
text_col = "clean_text"
features = []

In [3]:
class ModelConf(NamedTuple):
    name: str
    directory: Path
    oof_pred_file: Path
    model_max_length: int
    batch_size: int
    model_class: str = "auto"


models_conf: List[ModelConf] = [
    ModelConf(
        name="deberta_base", 
        directory=Path("models/aes2/deberta_v3_base/20240615_063400"),
        oof_pred_file = Path("models/aes2/deberta_v3_base/20240616_110724/oof.npy"),
        model_max_length=512,
        batch_size=128,
    ),
]

In [4]:
tim = scml.Timer()
tim.start()
percentiles=[.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]
os.environ["TOKENIZERS_PARALLELISM"] = "false"
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()
scml.seed_everything()

In [5]:
tra = pd.read_parquet(f"input/tra_{version}.parquet")
tra = tra.drop(columns=["source", "str_level"])
tra.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16441 entries, 0 to 16440
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   essay_id    16441 non-null  object
 1   score       16441 non-null  int8  
 2   clean_text  16441 non-null  object
dtypes: int8(1), object(2)
memory usage: 273.1+ KB


In [6]:
val = pd.read_parquet(f"input/val_{version}.parquet")
val = val.drop(columns=["source", "str_level"])
val.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 866 entries, 0 to 865
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   essay_id    866 non-null    object
 1   score       866 non-null    int8  
 2   clean_text  866 non-null    object
dtypes: int8(1), object(2)
memory usage: 14.5+ KB


# Base Model Inference

In [7]:
for mc in models_conf:
    print(mc.name)
    val[mc.name] = mylib.predict_holistic_score(
        ds=mylib.Aes2Dataset(
            tokenizer=AutoTokenizer.from_pretrained(mc.directory, model_max_length=mc.model_max_length),
            texts=val[text_col].tolist(),
        ),
        model=AutoModelForSequenceClassification.from_pretrained(mc.directory),
        batch_size=mc.batch_size,
        device=torch.device("cuda:1"),
        progress_bar=True,
    )
    with open(mc.oof_pred_file, "rb") as f:
        tra[mc.name] = np.load(f)
    features.append(mc.name)

deberta_base


/home/pankun/.local/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
predict hms score: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:10<00:00,  1.54s/it]


# Character & Word-level Features

In [8]:
#%%time
#col = "cw_len"
#tra[col] = tra[text_col].str.len()
#val[col] = val[text_col].str.len()
#features.append(col)

CPU times: user 4.42 ms, sys: 1.05 ms, total: 5.47 ms
Wall time: 5.38 ms


In [9]:
def digit_frac(row) -> float:
    return mylib.digit_frac(row[text_col])


def letter_frac(row) -> float:
    return mylib.letter_frac(row[text_col])


def space_frac(row) -> float:
    return mylib.space_frac(row[text_col])


def punc_frac(row) -> float:
    return mylib.punc_frac(row[text_col])


def upper_frac(row) -> float:
    return mylib.upper_frac(row[text_col])


def repeat_char_frac(row) -> float:
    return mylib.repeat_char_frac(row[text_col])


def repeat_substring_frac(row) -> float:
    return mylib.repeat_substring_frac(row[text_col])


def unique_word_frac(row) -> float:
    return mylib.unique_word_frac(row[text_col])


sf = mylib.StopwordFraction()


def stopword_frac(row) -> float:
    return sf(row[text_col])



cw_fns: List[Tuple[str, Callable]] = [
    ("cw_digit_frac", digit_frac),
    ("cw_letter_frac", letter_frac),
    ("cw_space_frac", space_frac),
    ("cw_punc_frac", punc_frac),
    ("cw_upper_frac", upper_frac),
    ("cw_repeat_char_frac", repeat_char_frac),
    ("cw_repeat_substring_frac", repeat_substring_frac),
    ("cw_unique_word_frac", unique_word_frac),
    ("cw_stopword_frac", stopword_frac),
]   
#for col, fn in cw_fns:
#    print(col)
#    tra[col] = tra.progress_apply(fn, axis=1)
#    val[col] = val.progress_apply(fn, axis=1)
#    features.append(col)
#tra[features] = tra[features].astype(np.float32)
#val[features] = val[features].astype(np.float32)

cw_digit_frac


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 866/866 [00:00<00:00, 17668.82it/s]


cw_letter_frac


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 866/866 [00:00<00:00, 16022.28it/s]


cw_space_frac


...

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 866/866 [00:34<00:00, 24.97it/s]


cw_unique_word_frac


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 866/866 [00:00<00:00, 38517.80it/s]


cw_stopword_frac


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 866/866 [00:00<00:00, 32684.26it/s]


# Textstat Features

In [10]:
def syllable_count(row) -> int:
    return textstat.syllable_count(row[text_col])


def lexicon_count(row) -> int:
    return textstat.lexicon_count(row[text_col])


def sentence_count(row) -> int:
    return textstat.sentence_count(row[text_col])


def syllables_per_word(row) -> float:
    return row["ts_syllable_count"] / (row["ts_lexicon_count"] + 1)


def syllables_per_sent(row) -> float:
    return row["ts_syllable_count"] / (row["ts_sentence_count"] + 1)


def words_per_sent(row) -> float:
    return row["ts_lexicon_count"] / (row["ts_sentence_count"] + 1)


def flesch_reading_ease(row) -> float:
    return textstat.flesch_reading_ease(row[text_col])


def flesch_kincaid_grade(row) -> float:
    return textstat.flesch_kincaid_grade(row[text_col])


def gunning_fog(row) -> float:
    return textstat.gunning_fog(row[text_col])


def smog_index(row) -> float:
    return textstat.smog_index(row[text_col])


def automated_readability_index(row) -> float:
    return textstat.automated_readability_index(row[text_col])


def coleman_liau_index(row) -> float:
    return textstat.coleman_liau_index(row[text_col])


def linsear_write_formula(row) -> float:
    return textstat.linsear_write_formula(row[text_col])


def dale_chall_readability_score(row) -> float:
    return textstat.dale_chall_readability_score(row[text_col])


def text_standard(row) -> float:
    return textstat.text_standard(row[text_col], float_output=True)


def mcalpine_eflaw(row) -> float:
    return textstat.mcalpine_eflaw(row[text_col])


textstat_fns: List[Tuple[str, Callable]] = [
    ("ts_syllable_count", syllable_count),
    ("ts_lexicon_count", lexicon_count),
    ("ts_sentence_count", sentence_count),
    ("ts_syllables_per_word", syllables_per_word),
    ("ts_syllables_per_sent", syllables_per_sent),
    ("ts_words_per_sent", words_per_sent),
    ("ts_flesch_reading_ease", flesch_reading_ease),
    ("ts_flesch_kincaid_grade", flesch_kincaid_grade),
    ("ts_gunning_fog", gunning_fog),
    ("ts_smog_index", smog_index),
    ("ts_automated_readability_index", automated_readability_index),
    ("ts_coleman_liau_index", coleman_liau_index),
    ("ts_linsear_write_formula", linsear_write_formula),
    ("ts_dale_chall_readability_score", dale_chall_readability_score),
    ("ts_text_standard", text_standard),
    ("ts_mcalpine_eflaw", mcalpine_eflaw),
]    
for col, fn in textstat_fns:
    print(col)
    tra[col] = tra.progress_apply(fn, axis=1)
    val[col] = val.progress_apply(fn, axis=1)
    if col.endswith("_count"):
        continue
    features.append(col)
tra[features] = tra[features].astype(np.float32)
val[features] = val[features].astype(np.float32)

ts_syllable_count


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 866/866 [00:00<00:00, 4559.27it/s]


ts_lexicon_count


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 866/866 [00:00<00:00, 30314.11it/s]


ts_sentence_count


...

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 866/866 [00:00<00:00, 3126.71it/s]


ts_text_standard


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 866/866 [00:00<00:00, 903.22it/s]


ts_mcalpine_eflaw


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 866/866 [00:00<00:00, 8237.46it/s]


# Review Data

In [11]:
tra[features] = tra[features].astype(np.float32)
val[features] = val[features].astype(np.float32)
tra = tra.drop(columns=["clean_text"])
val = val.drop(columns=["clean_text"])
tra.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16441 entries, 0 to 16440
Data columns (total 29 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   essay_id                         16441 non-null  object 
 1   score                            16441 non-null  int8   
 2   deberta_base                     16441 non-null  float32
 3   cw_len                           16441 non-null  float32
 4   cw_digit_frac                    16441 non-null  float32
 5   cw_letter_frac                   16441 non-null  float32
 6   cw_space_frac                    16441 non-null  float32
 7   cw_punc_frac                     16441 non-null  float32
 8   cw_upper_frac                    16441 non-null  float32
 9   cw_repeat_char_frac              16441 non-null  float32
 10  cw_repeat_substring_frac         16441 non-null  float32
 11  cw_unique_word_frac              16441 non-null  float32
 12  cw_stopword_frac  

In [12]:
val.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 866 entries, 0 to 865
Data columns (total 29 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   essay_id                         866 non-null    object 
 1   score                            866 non-null    int8   
 2   deberta_base                     866 non-null    float32
 3   cw_len                           866 non-null    float32
 4   cw_digit_frac                    866 non-null    float32
 5   cw_letter_frac                   866 non-null    float32
 6   cw_space_frac                    866 non-null    float32
 7   cw_punc_frac                     866 non-null    float32
 8   cw_upper_frac                    866 non-null    float32
 9   cw_repeat_char_frac              866 non-null    float32
 10  cw_repeat_substring_frac         866 non-null    float32
 11  cw_unique_word_frac              866 non-null    float32
 12  cw_stopword_frac      

In [13]:
tra.describe(percentiles=percentiles)

,score,deberta_base,cw_len,cw_digit_frac,cw_letter_frac,cw_space_frac,cw_punc_frac,cw_upper_frac,cw_repeat_char_frac,cw_repeat_substring_frac,cw_unique_word_frac,cw_stopword_frac,ts_syllable_count,ts_lexicon_count,ts_sentence_count,ts_syllables_per_word,ts_syllables_per_sent,ts_words_per_sent,ts_flesch_reading_ease,ts_flesch_kincaid_grade,ts_gunning_fog,ts_smog_index,ts_automated_readability_index,ts_coleman_liau_index,ts_linsear_write_formula,ts_dale_chall_readability_score,ts_text_standard,ts_mcalpine_eflaw
count,16441.000000,16441.000000,16441.000000,16441.000000,16441.000000,16441.000000,16441.000000,16441.000000,16441.000000,16441.000000,16441.000000,16441.000000,16441.000000,16441.000000,16441.000000,16441.000000,16441.000000,16441.000000,16441.000000,16441.000000,16441.000000,16441.000000,16441.000000,16441.000000,16441.000000,16441.000000,16441.000000,16441.000000
mean,2.948422,2.948711,2045.229858,0.002810,0.795705,0.180791,0.020694,0.018777,0.015127,0.002631,0.536852,0.501528,505.336975,367.427399,19.512318,1.364587,25.998779,19.002592,70.220360,8.560081,10.212701,10.118253,10.236975,8.303247,10.233084,7.541533,9.192932,28.973743
std,1.044865,0.921251,865.908569,0.003790,0.011588,0.010141,0.006730,0.019409,0.004413,0.008531,0.074685,0.048731,216.673706,150.410385,8.519920,0.092320,10.216710,7.685802,14.071000,4.615396,4.695395,1.846192,5.846623,1.790902,4.973484,1.007713,3.714410,16.534847
min,1.000000,0.648224,711.000000,0.000000,0.725769,0.145091,0.000000,0.000000,0.000890,0.000000,0.056763,0.000000,175.000000,150.000000,1.000000,1.083582,8.653846,7.115385,-434.510010,1.300000,3.670000,0.000000,0.400000,1.190000,2.600000,1.240000,0.000000,10.400000
1%,1.000000,1.181190,843.400000,0.000000,0.764987,0.160097,0.006645,0.001748,0.006180,0.000000,0.370033,0.392971,206.000000,158.000000,4.000000,1.166148,13.270629,10.260819,36.812000,3.400000,5.414000,6.200000,3.700000,4.230000,4.300000,5.774000,5.000000,15.200000
5%,1.000000,1.528137,968.000000,0.000000,0.775773,0.165006,0.010506,0.007220,0.008368,0.000000,0.417204,0.425197,238.000000,177.000000,8.000000,1.218045,15.952381,12.130435,52.459999,4.700000,6.510000,7.300000,5.300000,5.360000,5.375000,6.190000,6.000000,17.900000
10%,2.000000,1.800045,1076.000000,0.000000,0.780418,0.168097,0.012575,0.009241,0.009689,0.000000,0.442029,0.442623,264.000000,198.000000,10.000000,1.247761,17.642857,13.192307,57.299999,5.300000,7.150000,7.900000,6.200000,5.970000,6.000000,6.430000,6.000000,19.500000
20%,2.000000,2.105222,1285.000000,0.000000,0.786397,0.171932,0.015152,0.011550,0.011324,0.000000,0.472527,0.462857,316.000000,235.000000,12.000000,1.285047,19.652174,14.588235,62.070000,6.200000,7.940000,8.700000,7.300000,6.780000,6.857143,6.760000,7.000000,21.600000
30%,2.000000,2.383826,1486.000000,0.000000,0.790305,0.175000,0.016987,0.013314,0.012605,0.000000,0.495512,0.476886,365.000000,270.000000,14.000000,1.313846,21.391304,15.714286,65.559998,6.900000,8.540000,9.200000,8.200000,7.320000,7.571429,7.010000,8.000000,23.299999
40%,3.000000,2.626459,1685.000000,0.000779,0.793571,0.177674,0.018700,0.015110,0.013782,0.001419,0.516055,0.489627,415.000000,306.000000,16.000000,1.337802,22.900000,16.714285,68.599998,7.500000,9.090000,9.600000,8.900000,7.830000,8.166667,7.240000,8.000000,24.900000


In [14]:
val.describe(percentiles=percentiles)

,score,deberta_base,cw_len,cw_digit_frac,cw_letter_frac,cw_space_frac,cw_punc_frac,cw_upper_frac,cw_repeat_char_frac,cw_repeat_substring_frac,cw_unique_word_frac,cw_stopword_frac,ts_syllable_count,ts_lexicon_count,ts_sentence_count,ts_syllables_per_word,ts_syllables_per_sent,ts_words_per_sent,ts_flesch_reading_ease,ts_flesch_kincaid_grade,ts_gunning_fog,ts_smog_index,ts_automated_readability_index,ts_coleman_liau_index,ts_linsear_write_formula,ts_dale_chall_readability_score,ts_text_standard,ts_mcalpine_eflaw
count,866.000000,866.000000,866.000000,866.000000,866.000000,866.000000,866.000000,866.000000,866.000000,866.000000,866.000000,866.000000,866.000000,866.000000,866.000000,866.000000,866.000000,866.000000,866.000000,866.000000,866.000000,866.000000,866.000000,866.000000,866.000000,866.000000,866.00000,866.000000
mean,2.948037,2.921389,2060.635010,0.002611,0.795851,0.181177,0.020361,0.018148,0.015119,0.002907,0.533583,0.504496,509.431885,370.844116,19.625866,1.362833,26.972654,19.788095,68.661377,9.188222,10.897321,10.107390,11.009238,8.253776,10.499711,7.593972,9.80485,31.437298
std,1.046139,0.929846,849.165894,0.003620,0.011515,0.010704,0.006710,0.009073,0.004385,0.007891,0.075234,0.048059,212.012878,147.202759,8.809137,0.092970,20.295984,15.702071,32.334080,12.169928,12.454148,1.943423,15.534221,1.855127,5.296445,1.734166,12.05920,45.399860
min,1.000000,0.971314,732.000000,0.000000,0.743163,0.155241,0.000000,0.000000,0.004212,0.000000,0.264228,0.354054,190.000000,151.000000,1.000000,1.108434,10.962963,8.500000,-628.880005,2.200000,3.910000,0.000000,2.800000,3.190000,2.800000,5.380000,0.00000,12.100000
1%,1.000000,1.110007,843.000000,0.000000,0.765810,0.159538,0.006005,0.000906,0.006511,0.000000,0.361339,0.385154,205.600000,159.300000,4.000000,1.153609,13.469216,10.195938,37.691500,3.265000,5.416500,5.765000,3.565000,3.959500,4.235606,5.800000,5.00000,14.765000
5%,1.000000,1.395183,969.000000,0.000000,0.776374,0.164993,0.010542,0.006928,0.008457,0.000000,0.410804,0.427421,238.500000,180.000000,8.000000,1.208756,16.406060,12.396826,53.077499,4.800000,6.655000,7.400000,5.400000,5.155000,5.458333,6.172500,6.00000,18.425000
10%,2.000000,1.696749,1087.500000,0.000000,0.781379,0.168216,0.012364,0.009094,0.009790,0.000000,0.439701,0.446998,269.500000,202.000000,9.500000,1.241224,17.884259,13.362500,57.400002,5.400000,7.280000,7.900000,6.150000,5.840000,6.000000,6.400000,6.00000,19.900000
20%,2.000000,2.021870,1314.000000,0.000000,0.786875,0.171930,0.015158,0.011138,0.011118,0.000000,0.466775,0.467442,322.000000,241.000000,12.000000,1.284289,19.769230,14.709678,61.900002,6.300000,8.080000,8.700000,7.300000,6.660000,6.857143,6.710000,7.00000,21.700001
30%,2.000000,2.393944,1482.000000,0.000000,0.790532,0.174986,0.016810,0.013238,0.012684,0.000778,0.493053,0.479837,364.500000,274.000000,14.000000,1.314428,21.358655,15.666667,65.730003,6.900000,8.570000,9.300000,8.100000,7.245000,7.666667,6.970000,7.00000,23.400000
40%,3.000000,2.683454,1700.000000,0.000835,0.793531,0.177907,0.018519,0.014957,0.013810,0.001460,0.514706,0.492891,420.000000,310.000000,16.000000,1.339181,22.853659,16.727272,68.599998,7.400000,9.100000,9.700000,8.900000,7.770000,8.333333,7.230000,8.00000,24.799999


In [15]:
features.sort()
with open(f"output/features_{version}.json", "w") as f:
    json.dump({"feature_names": features}, f)

In [16]:
tra.to_parquet(f"output/mtra_{version}.parquet", index=False)
val.to_parquet(f"output/mval_{version}.parquet", index=False)
assert tra.notna().all(axis=None)
assert val.notna().all(axis=None)

In [17]:
tim.stop()
print(f"Total time taken {str(tim.elapsed)}")

Total time taken 0:12:59.580492
